# JULIA MPI First Example: pi computaton

First step was to load MPI on my mac.  Seems mpich and openmpi are two reasonable choices
with probably no beginner's reason to prefer one over the other. <br>

I did  <t> brew install gcc </t> first to get the gcc compiler.  I ran into problems.  
The magic thing that told me what to do was <t> brew doctor </t>.  It wanted me to type
<t> xcode-select --install </t> and when I did, all was good.  I then typed
<t> brew install mpich </t> and mpi was just working.

My first example was to reproduce <a href="http://www.mcs.anl.gov/research/projects/mpi/tutorial/mpiexmpl/src/pi/C/solution.html">
the classic mypi </a> in the notebook

In [1]:
# Pkg.add("MPI")

In [1]:
using MPI

In [2]:
m = MPIManager(np=8)

MPI.MPIManager(8,Dict{Int64,Int64}(),Dict{Int64,Int64}(),0,`mpirun -np 8`,60,false,Condition(Any[]),false,0x2af8,IO[],#undef,#undef)

In [3]:
addprocs(m)
#@mpi_do m comm = MPI.COMM_WORLD

8-element Array{Int64,1}:
 2
 3
 4
 5
 6
 7
 8
 9

In [4]:
@mpi_do m comm = MPI.COMM_WORLD
#
# Enter number of intervals, and tell every processor
# Traditional MPI would do this with a BCAST
@mpi_do m n=45

In [5]:
# Let's see if the processors got it
@mpi_do m println(n)

	From worker 3:	45
	From worker 7:	45
	From worker 4:	45
	From worker 8:	45
	From worker 2:	45
	From worker 9:	45
	From worker 6:	45
	From worker 5:	45


In [6]:
# my MPI id
@mpi_do m myid = MPI.Comm_rank(comm)
@mpi_do m println(myid)

	From worker 4:	2
	From worker 7:	5
	From worker 2:	0
	From worker 9:	7
	From worker 3:	1
	From worker 8:	6
	From worker 5:	3
	From worker 6:	4


In [7]:
# Get the number of processors
@mpi_do m np=MPI.Comm_size(comm)
@mpi_do m println(np)

	From worker 7:	8
	From worker 2:	8
	From worker 9:	8
	From worker 4:	8
	From worker 3:	8
	From worker 5:	8
	From worker 8:	8
	From worker 6:	8


Compute $\int_0^1 4/(1+x^2) dx= 4 atan(x)]_0^1$ which evaluates to $\pi$

In [8]:
@time @mpi_do m (
   n = 50_000_000;
   comm = MPI.COMM_WORLD;
   s=0.0;
   for i= MPI.Comm_rank(comm)+1 :  MPI.Comm_size(comm) : n 
    x = (i-.5)/n 
    s += 4/(1+x^2) 
   end;
   mypi = s/n;
   our_π = MPI.Reduce(mypi, MPI.SUM, 0, comm);
   if myid==0
      println(our_π);
      println(our_π - π); 
   end
)




	From worker 2:	3.1415926535897043
	From worker 2:	-8.881784197001252e-14
  3.965622 seconds (32.83 k allocations: 1.540 MB)


In [9]:
@time (n = 50_000_000;
h=1/n;
 our_π =0;
for i=0:h:1
    our_π += 4/(1+i^2)
end;
#our_π*h
)

 11.396405 seconds (350.06 M allocations: 5.963 GB, 4.10% gc time)


In [ ]:
3.7*4